In [3]:
!pip install awswrangler

Keyring is skipped due to an exception: 'keyring.backends'
  Using cached awswrangler-2.17.0-py3-none-any.whl (251 kB)
  Using cached opensearch_py-2.0.0-py2.py3-none-any.whl (204 kB)
  Using cached jsonpath_ng-1.5.3-py3-none-any.whl (29 kB)
  Using cached requests_aws4auth-1.1.2-py2.py3-none-any.whl (24 kB)
  Using cached pyarrow-8.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (29.3 MB)
  Using cached gremlinpython-3.6.1-py2.py3-none-any.whl (73 kB)
  Using cached pg8000-1.29.3-py3-none-any.whl (51 kB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached redshift_connector-2.0.909-py3-none-any.whl (112 kB)
  Using cached progressbar2-4.2.0-py2.py3-none-any.whl (27 kB)
  Using cached aiohttp-3.8.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached aenum-3.1.11-py3-none-any.wh

In [4]:
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np
import re
from sagemaker import get_execution_role
import matplotlib.pyplot as plt
from tqdm import tqdm 

role = get_execution_role()
s3 = boto3.resource('s3')

In [5]:
s3 = boto3.resource('s3')
my_bucket = s3.Bucket('icusics-db')

tables = []
for my_bucket_object in my_bucket.objects.all():
    tables.append(my_bucket_object.key)
tables = pd.Series(tables)

In [6]:
variables = wr.s3.read_parquet(path="s3://icusics-db/d_variables/d_variables.parquet")
variables_h4 = variables[variables.hospital_coded==4]
diagnoses = wr.s3.read_parquet(path="s3://icusics-db/diagnoses/diagnoses_h4.parquet")
labresults_numeric = wr.s3.read_parquet(path="s3://icusics-db/labresults_numeric/labresults_numeric_h4.parquet")

# Check for predictors variable_id

Predictors:
 - Albumine
 - Potassium
 - SpO2
 - Haematocrit
 - Ph
 - Blood Urine Nitrate / Urea * 0.357
 - Diabetes
 - Chronic Kidney Disease
 - Hyper tension
 - Heart failure

Outcome:
 - CRRT

In [5]:
# Albumine
idx = variables_h4.name.str.lower().str.extract(r'(.*\wlb\w+.*)').dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
15960,4,labresults_numeric,4024001892,NaN,<NA>,Albúmina-Plasma,Albumina-P,<NA>
16004,4,labresults_numeric,4024002303,NaN,<NA>,Prealbúmina-Sèrum,Prealbumina,<NA>
16011,4,labresults_numeric,4024002341,NaN,<NA>,Albumina-Sèrum,Albumina,<NA>


In [20]:
# Albumine
idx = variables.name.str.lower().str.extract(r'(.*\wlb\w+.*)').dropna().index.tolist()
variables.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
224,1,labresults_numeric,1024000512,NaN,<NA>,Albúmina-Sèrum,Albúmina-S,<NA>
363,1,labresults_numeric,1024000684,NaN,<NA>,S-Albumina,S-Albumina,<NA>
383,1,labresults_numeric,1024000708,NaN,<NA>,LCR-Albúmina,LCR-Albúmina,<NA>
4165,1,observed_numeric,1015001698,NaN,<NA>,Albúmina,Albúmina,EMINA
5374,2,labresults_numeric,2024000340,NaN,<NA>,albúmina sérica,Albúmina,<NA>
5423,2,labresults_numeric,2024000655,NaN,<NA>,Prealbúmina,Prealbúmina,<NA>
10211,3,labresults_numeric,3024000882,NaN,<NA>,Prealbúmina,Prealbúmina,<NA>
10295,3,labresults_numeric,3024001123,NaN,<NA>,Uri-Quocient Albúmin/Creatinin,O-Alb/Creat,<NA>
10306,3,labresults_numeric,3024001137,NaN,<NA>,Microalbuminuria-Orina 24 h,O24hMicroalb,<NA>
10307,3,labresults_numeric,3024001802,NaN,<NA>,Srm-Albúmina,Albúmina,<NA>


In [6]:
# Potassium
idx = variables_h4.name.str.lower().str.extract(r'(.*\bpotas\w+.*)').dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
15791,4,derived_numeric,4030005310,NaN,<NA>,Cum potassium intake,IN K/c,<NA>
15834,4,derived_numeric,4030014580,NaN,<NA>,APACHE 2 serum potassium,A2 S-K,APACHE II serum potassium
15842,4,derived_numeric,4030014660,NaN,<NA>,S2 serum potassium,S2 S-K,SAPS II serum potassium
15872,4,derived_numeric,4030015310,NaN,<NA>,Daily potassium intake,IN K/d,<NA>
15911,4,labresults_numeric,4020000500,NaN,<NA>,Potassi Sèrum,Potassi,<NA>
15981,4,labresults_numeric,4024002036,NaN,<NA>,Potassi-Plasma,Potassi-P,<NA>
21257,4,observed_numeric,4010014595,NaN,<NA>,Min 24 h serum potassium,S-K min,Validated minimum 24 h serum potassium
21258,4,observed_numeric,4010014600,NaN,<NA>,Max 24 h serum potassium,S-K max,Validated maximum 24 h serum potassium


In [7]:
# SpO2
idx = variables_h4.name.str.lower().str.extract(r'(.*(\bspo2\b|\bsatura\w+).*)')[0].dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
15766,4,derived_numeric,4030003500,NaN,<NA>,SpO2/FiO2 ratio,SpO2/FiO2,"= SpO2 / FiO2 ; SpO2 %, FiO2 %/100 (2 h)"
16047,4,labresults_numeric,4024002588,NaN,<NA>,Index saturació transferrina,Sat transfer,<NA>
16254,4,monitored_numeric,4000004000,NaN,<NA>,SpO2,SpO2,SatO2 perifèrica
16256,4,monitored_numeric,4000004311,NaN,<NA>,Saturació O2 Regió 1,Sr1O2,<NA>
16257,4,monitored_numeric,4000004312,NaN,<NA>,Saturació O2 Regió 2,Sr2O2,<NA>
16258,4,monitored_numeric,4000004313,NaN,<NA>,Saturació O2 Regió 3,Sr3O2,<NA>
16259,4,monitored_numeric,4000004314,NaN,<NA>,Saturació O2 Regió 4,Sr4O2,<NA>
21350,4,observed_numeric,4015003101,NaN,<NA>,Saturació d'hemoglobina,SatHb preIET,abans de la intubació traqueal


In [8]:
# Haematocrit
idx = variables_h4.name.str.lower().str.extract(r'(.*ha?emat\w+.*)')[0].dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
15828,4,derived_numeric,4030014540,NaN,<NA>,APACHE 2 hematocrit,A2 B-Hcr,APACHE II hematocrit
15907,4,labresults_numeric,4020000100,NaN,<NA>,Hematòcrit,Hematòcrit,<NA>
15999,4,labresults_numeric,4024002245,NaN,<NA>,Hematies,Hematies,<NA>
16750,4,observed_categoric,4014000210,1.0,No,Hematologic malignancy,Hem maligncy,SAPS II and SAPS 3 comorbid condition
16751,4,observed_categoric,4014000210,2.0,Yes,Hematologic malignancy,Hem maligncy,SAPS II and SAPS 3 comorbid condition
17470,4,observed_categoric,4015002317,1.0,-,Hematúria,Sang orina,<NA>
17471,4,observed_categoric,4015002317,2.0,+,Hematúria,Sang orina,<NA>
17472,4,observed_categoric,4015002317,3.0,++,Hematúria,Sang orina,<NA>
17473,4,observed_categoric,4015002317,4.0,+++,Hematúria,Sang orina,<NA>
17474,4,observed_categoric,4015002317,5.0,+ Eritròcits,Hematúria,Sang orina,<NA>


In [9]:
# Ph
idx = variables_h4.name.str.lower().str.extract(r'(.*ph.*)')[0].dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
15832,4,derived_numeric,4030014560,NaN,<NA>,APACHE 2 art pH,A2 pHa,APACHE II arterial pH
15931,4,labresults_numeric,4024001491,NaN,<NA>,pH,pH,<NA>
15949,4,labresults_numeric,4024001725,NaN,<NA>,Ven-pH,Ven-pH,<NA>
16061,4,labresults_numeric,4024002679,NaN,<NA>,pH(T),pH(T),<NA>
16260,4,monitored_numeric,4000004510,NaN,<NA>,Gastric intramucosal pH,pHi,pH intramucosal gàstric
17459,4,observed_categoric,4015002315,1.0,5,pH orina,pH orina,<NA>
17460,4,observed_categoric,4015002315,2.0,6,pH orina,pH orina,<NA>
17461,4,observed_categoric,4015002315,3.0,"6,5",pH orina,pH orina,<NA>
17462,4,observed_categoric,4015002315,4.0,7,pH orina,pH orina,<NA>
17463,4,observed_categoric,4015002315,5.0,8,pH orina,pH orina,<NA>


In [10]:
# Blood Urine Nitrate
# urea * 0.357
idx = variables_h4.name.str.lower().str.extract(r'(.*urea.*)')[0].dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
15839,4,derived_numeric,4030014630,NaN,<NA>,S2 serum urea,S2 S-Urea,SAPS II serum urea
15919,4,labresults_numeric,4020004100,NaN,<NA>,Urea Sèrum,Urea,<NA>
15989,4,labresults_numeric,4024002104,NaN,<NA>,Urea-Plasma,Urea-P,<NA>
21265,4,observed_numeric,4010014635,NaN,<NA>,Max 24 h serum urea,S-Urea max,Validated maximum 24 h serum urea


In [12]:
# Diabetes
idx = diagnoses.referencecodename.str.extract(r'(.*[Dd]iabe.*)').dropna().index.tolist()
diabetes_codes = diagnoses.loc[idx, ['referencecode','referencecodename']].drop_duplicates().referencecode
diagnoses.loc[idx, ['referencecode','referencecodename']].drop_duplicates()

,referencecode,referencecodename
5,E13.9,Diabetis mellitus
91,E11,Diabetis mellitus de tipus 2
385,E11.59,Diabetis mellitus de tipus 2 amb altres compli...
528,E11.65,Diabetis mellitus de tipus 2 amb hiperglucèmia
753,E11.01,Diabetis mellitus de tipus 2 amb hiperosmolari...
1613,249.00,Diabetis mellitus
1712,E13.9,Altres tipus especificats de diabetis mellitus...
1791,E23.2,Diabetis insípida
2014,E13.42,Altres tipus especificats de diabetis mellitus...
2256,E11.641,"Diabetis mellitus de tipus 2 amb hipoglucèmia,..."


In [13]:
# Chronic Kidney Disease
idx = diagnoses.referencecodename.str.lower().str.extract(r'(.*renal cr\wnica.*)').dropna().index.tolist()
ckd_codes = diagnoses.loc[idx, ['referencecode','referencecodename']].drop_duplicates().referencecode
diagnoses.loc[idx, ['referencecode','referencecodename']].drop_duplicates()

,referencecode,referencecodename
167,N18.9,Insuficiència renal crònica
474,N18,Malaltia renal crònica
1103,N18.5,"Malaltia renal crònica, estadi 5"
1305,N18.9,Malaltia renal crònica no especificada
3598,N18.2,"Malaltia renal crònica, estadi 2 (lleu)"
3790,585.9,Insuficiència renal crònica
4866,N18.4,"Malaltia renal crònica, estadi 4 (greu)"
5203,I13.1,Cardiopatia hipertensiva i mal. renal crònica ...
5219,D63.1,Anèmia en malaltia renal crònica
6570,E13.22,Altres tipus especificats de diabetis mellitus...


In [11]:
# Hyper tension
idx = diagnoses.referencecodename.str.lower().str.extract(r'(.*hipertens.*)').dropna().index.tolist()
diagnoses.loc[idx, ['referencecode','referencecodename']].drop_duplicates()#.referencecodename.value_counts()

,referencecode,referencecodename
6,I10,Hipertensió essencial
9,I16.9,Crisi hipertensiva no especificada
10,I10,Hipertensió (primària) essencial
274,G93.5,Hipertensió intracranial
464,401.9,Hipertensió essencial
585,K76.6,Hipertensió portal
737,401.1,Hipertensió essencial benigna
1099,I15.8,Altres tipus d´hipertensió secundària
1167,I27.2,Altres tipus d´hipertensió pulmonar secundària
1547,I27.0,Hipertensió pulmonar primària


In [14]:
# Heart failure
idx = diagnoses.referencecodename.str.lower().str.extract(r'(.*insufici\wncia card\waca.*)').dropna().index.tolist()
diagnoses.loc[idx, ['referencecode','referencecodename']].drop_duplicates()#.referencecodename.value_counts()

,referencecode,referencecodename
94,I50,Insuficiència cardíaca
176,I50.9,Insuficiència cardíaca congestiva
1206,428.0,Insuficiència cardíaca congestiva
2360,402.11,Cardiopatia hipertensiva benigna a/insuficiènc...
5050,I50.2,Insuficiència cardíaca (congestiva) sistòlica
5203,I13.1,Cardiopatia hipertensiva i mal. renal crònica ...
5837,I50.9,Insuficiència cardíaca no especificada
7921,I11.9,Cardiopatia hipertensiva sense insuficiència c...
9330,I11.0,Cardiopatia hipertensiva amb insuficiència car...


In [15]:
diagnosis_codes = [
    # diabetes_codes
    # ckd_codes
    'I10','401.1','402.11','I15.8','I15.0','I11','I15','I13.1','I11.9','I15.1','I11.0','I11.0','401.9' # hyptertension
    'I50','I50.9','428.0','402.11','I50.2','I50.9','I11.0' # heart failure
] + list(diabetes_codes) + list(ckd_codes)

comorbidities = diagnoses[diagnoses.referencecode.isin(diagnosis_codes)]
comorbidities

,a_patientid,diag_type,referencecode,referencecodename
5,4000555,secondary,E13.9,Diabetis mellitus
6,4000555,secondary,I10,Hipertensió essencial
10,4000561,secondary,I10,Hipertensió (primària) essencial
19,4001427,secondary,E13.9,Diabetis mellitus
25,4001674,secondary,I10,Hipertensió essencial
...,...,...,...,...
12673,4999230,secondary,N18.9,Malaltia renal crònica no especificada
12681,4999848,secondary,E11,Diabetis mellitus de tipus 2
12686,4999848,secondary,N18.9,Insuficiència renal crònica
12690,4999872,secondary,I10,Hipertensió essencial


In [16]:
# CRRT
idx = variables_h4.name.str.lower().str.extract(r'(.*\bfl\wx.*)')[0].dropna().index.tolist()
variables_h4.loc[idx]

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
16079,4,monitored_categoric,4000003824,1.0,ACCEL,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16080,4,monitored_categoric,4000003824,2.0,SQUARE,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16081,4,monitored_categoric,4000003824,3.0,50%DEC,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16082,4,monitored_categoric,4000003824,4.0,DECEL,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16083,4,monitored_categoric,4000003824,5.0,50%ACCEL,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16084,4,monitored_categoric,4000003824,6.0,MOD.SINE,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16085,4,monitored_categoric,4000003824,7.0,SINE,Patró de fluxe Hamilton,Hamilton Pat,Patró de fluxe Hamilton
16142,4,monitored_categoric,4000009541,6.0,RAMP,Tipus Corba Flux Inspiratori,PB840 Ptrn,<NA>
16143,4,monitored_categoric,4000009541,7.0,SQUARE,Tipus Corba Flux Inspiratori,PB840 Ptrn,<NA>
16217,4,monitored_numeric,4000001380,NaN,<NA>,Flux Bomba de sang,Flux Sang,Flux bomba de sang configurada


In [7]:
lab_items = [
    4024002341, # Albumine
    4020000500, # Potassium
    4020000100, # Haematocrit
    4024001491, # Ph
    4020004100, # BUN
]

lab_predictors = labresults_numeric[labresults_numeric.a_variableid.isin(lab_items)]
lab_predictors = lab_predictors.merge(variables.loc[:, ['a_variableid','abbreviation']], how='left', on='a_variableid')
lab_predictors

,a_patientid,a_variableid,time,value,abbreviation
0,4004981,4020000100,-478,39.30,Hematòcrit
1,4004981,4020000100,343,28.20,Hematòcrit
2,4004981,4020000100,1821,28.50,Hematòcrit
3,4004981,4020000100,3161,29.30,Hematòcrit
4,4004981,4020000100,4581,28.60,Hematòcrit
...,...,...,...,...,...
178914,4991748,4020000100,922,44.50,Hematòcrit
178915,4991748,4020000500,922,4.23,Potassi
178916,4991748,4020004100,932,17.00,Urea
178917,4991748,4024001491,81,7.34,pH


In [18]:
tmp_path = './tmp/labs.csv'
#lab_predictors.to_csv(tmp_path)
lab_predictors = pd.read_csv(tmp_path, index_col=0)
lab_predictors

,a_patientid,a_variableid,time,value,abbreviation
0,4004981,4020000100,-478,39.30,Hematòcrit
1,4004981,4020000100,343,28.20,Hematòcrit
2,4004981,4020000100,1821,28.50,Hematòcrit
3,4004981,4020000100,3161,29.30,Hematòcrit
4,4004981,4020000100,4581,28.60,Hematòcrit
...,...,...,...,...,...
178914,4991748,4020000100,922,44.50,Hematòcrit
178915,4991748,4020000500,922,4.23,Potassi
178916,4991748,4020004100,932,17.00,Urea
178917,4991748,4024001491,81,7.34,pH


In [19]:
vital_items = [
    4000004000, # SpO2
]

outcome_items = [
    4000001380, # Flux Sang
    4000005666, # Flux Efluent
]

monitored_predictors = []
outcomes = []

monitored_tables = tables[tables.str.match(r'monitored_numeric/monitored_numeric_4')]
for table in tqdm(monitored_tables):
    t_ith = wr.s3.read_parquet(path=f"s3://icusics-db/{table}")
    v_ith = t_ith[t_ith.a_variableid.isin(vital_items)]
    o_th = t_ith[t_ith.a_variableid.isin(outcome_items)]
    monitored_predictors.append(v_ith)
    outcomes.append(o_th)
    
monitored_predictors = pd.concat(monitored_predictors, ignore_index=True)
outcomes = pd.concat(outcomes, ignore_index=True)
outcomes

100%|██████████| 100/100 [00:59<00:00,  1.69it/s]


,a_patientid,a_variableid,time,value
0,4003958,4000001380,64497,70.0
1,4003958,4000001380,64512,70.0
2,4003958,4000001380,64527,70.0
3,4003958,4000001380,64542,70.0
4,4003958,4000001380,64572,70.0
...,...,...,...,...
267007,4995531,4000001380,470,200.0
267008,4995531,4000005666,306,3050.0
267009,4995531,4000005666,470,3050.0
267010,4997421,4000001380,1965,180.0


In [21]:
outcomes = outcomes.merge(variables.loc[:, ['a_variableid','abbreviation']], how='left', on='a_variableid')
tmp_path = './tmp/outcomes.csv'
outcomes.to_csv(tmp_path)
outcomes = pd.read_csv(tmp_path, index_col=0)
outcomes

,a_patientid,a_variableid,time,value,abbreviation
0,4003958,4000001380,64497,70.0,Flux Sang
1,4003958,4000001380,64512,70.0,Flux Sang
2,4003958,4000001380,64527,70.0,Flux Sang
3,4003958,4000001380,64542,70.0,Flux Sang
4,4003958,4000001380,64572,70.0,Flux Sang
...,...,...,...,...,...
267007,4995531,4000001380,470,200.0,Flux Sang
267008,4995531,4000005666,306,3050.0,Flux Efluent
267009,4995531,4000005666,470,3050.0,Flux Efluent
267010,4997421,4000001380,1965,180.0,Flux Sang


In [22]:
#monitored_predictors = monitored_predictors.merge(variables.loc[:, ['a_variableid','abbreviation']], how='left', on='a_variableid')
tmp_path = './tmp/vitals.csv'
#monitored_predictors.to_csv(tmp_path)
monitored_predictors = pd.read_csv(tmp_path, index_col=0)
monitored_predictors

,a_patientid,a_variableid,time,value,abbreviation
0,4000318,4000004000,8,100.0,SpO2
1,4000318,4000004000,9,100.0,SpO2
2,4000318,4000004000,11,100.0,SpO2
3,4000318,4000004000,15,100.0,SpO2
4,4000318,4000004000,19,100.0,SpO2
...,...,...,...,...,...
10175954,4999880,4000004000,980,97.0,SpO2
10175955,4999880,4000004000,982,97.0,SpO2
10175956,4999880,4000004000,986,97.0,SpO2
10175957,4999880,4000004000,992,97.0,SpO2


In [23]:
lower_bound = 12 * 60
upper_bound = 5 * 24 * 60

labs = lab_predictors.query(f"time<={lower_bound}")
vitals = monitored_predictors.query(f"time<={lower_bound}")

vitals = pd.pivot_table(
    vitals,
    index='a_patientid',
    columns='abbreviation',
    values='value',
    aggfunc=[min,max]
).reset_index()

labs = pd.pivot_table(
    labs,
    index='a_patientid',
    columns='abbreviation',
    values='value',
    aggfunc=[min,max]
).reset_index()

labs

# features = vitals.merge(labs, how='inner', on='a_patientid')

a_patientid      min                                      max  \
abbreviation             Albumina Hematòcrit Potassi   Urea    pH Albumina   
0                4000318      NaN       39.5     NaN    NaN  7.35      NaN   
1                4000422      3.9       34.3    4.09   83.0  7.16      3.9   
2                4000555      NaN       49.6     NaN    NaN  7.26      NaN   
3                4000561      4.5       51.3    3.97   27.0  7.38      4.5   
4                4000592      2.9       36.7    4.00   67.0  7.27      2.9   
...                  ...      ...        ...     ...    ...   ...      ...   
3820             4999651      NaN       30.5     NaN    NaN  7.42      NaN   
3821             4999848      NaN       28.5    4.63   97.0  7.32      NaN   
3822             4999872      NaN       32.0    3.25   25.0  7.44      NaN   
3823             4999880      2.9       24.1    5.61  122.0  7.34      3.1   
3824             4999885      NaN       42.1     NaN    NaN  7.40      NaN   

                                              
abbreviation Hematòcrit Potassi   Urea    pH  
0                  39.5     NaN    NaN  7.35  
1                  34.3    4.09   83.0  7.17  
2                  49.6     NaN    NaN  7.44  
3                  51.3    3.97   27.0  7.40  
4                  41.9    4.57   68.0  7.43  
...                 ...     ...    ...   ...  
3820               30.5     NaN    NaN  7.44  
3821               32.6    4.92  100.0  7.34  
3822               32.0    3.25   25.0  7.44  
3823               26.6    5.80  127.0  7.34  
3824               42.1     NaN    NaN  7.42  

[3825 rows x 11 columns]

In [24]:
vitals

,a_patientid,min,max
abbreviation,,SpO2,SpO2
0,4000318,91.3,100.0
1,4000422,91.0,99.0
2,4000555,93.3,100.0
3,4000561,92.9,100.0
4,4000592,73.9,100.0
...,...,...,...
3898,4999651,97.1,100.0
3899,4999848,89.1,100.0
3900,4999872,96.0,100.0


In [25]:
labs.columns = list(map(lambda t: t[0] + '_' + t[1], labs.columns))
vitals.columns = list(map(lambda t: t[0] + '_' + t[1], vitals.columns))

features = vitals.merge(labs, how='inner', on='a_patientid_').rename(columns={'a_patientid_': 'a_patientid'})
features

,a_patientid,min_SpO2,max_SpO2,min_Albumina,min_Hematòcrit,min_Potassi,min_Urea,min_pH,max_Albumina,max_Hematòcrit,max_Potassi,max_Urea,max_pH
0,4000318,91.3,100.0,NaN,39.5,NaN,NaN,7.35,NaN,39.5,NaN,NaN,7.35
1,4000422,91.0,99.0,3.9,34.3,4.09,83.0,7.16,3.9,34.3,4.09,83.0,7.17
2,4000555,93.3,100.0,NaN,49.6,NaN,NaN,7.26,NaN,49.6,NaN,NaN,7.44
3,4000561,92.9,100.0,4.5,51.3,3.97,27.0,7.38,4.5,51.3,3.97,27.0,7.40
4,4000592,73.9,100.0,2.9,36.7,4.00,67.0,7.27,2.9,41.9,4.57,68.0,7.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3804,4999651,97.1,100.0,NaN,30.5,NaN,NaN,7.42,NaN,30.5,NaN,NaN,7.44
3805,4999848,89.1,100.0,NaN,28.5,4.63,97.0,7.32,NaN,32.6,4.92,100.0,7.34
3806,4999872,96.0,100.0,NaN,32.0,3.25,25.0,7.44,NaN,32.0,3.25,25.0,7.44
3807,4999880,94.0,98.0,2.9,24.1,5.61,122.0,7.34,3.1,26.6,5.80,127.0,7.34


In [26]:
comorbidities['diabetis_flag'] = comorbidities.referencecodename.str.lower().str.match(r'.*diabet.*').astype(int)
comorbidities['ckd_flag'] = comorbidities.referencecodename.str.lower().str.match(r'.*renal cr.nica.*').astype(int)
comorbidities['hyperTension_flag'] = comorbidities.referencecodename.str.lower().str.match(r'.*hipertens.*').astype(int)
comorbidities['heartFailure_flag'] = comorbidities.referencecodename.str.lower().str.match(r'.*insufici\wncia card\waca.*').astype(int)

df_comorb = comorbidities.groupby('a_patientid').agg(dict.fromkeys(comorbidities.columns[-4:], 'max')).reset_index()
df_comorb

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,a_patientid,diabetis_flag,ckd_flag,hyperTension_flag,heartFailure_flag
0,4000555,1,0,1,0
1,4000561,0,0,1,0
2,4001427,1,0,0,0
3,4001674,0,0,1,0
4,4004551,1,0,1,0
...,...,...,...,...,...
880,4997421,1,1,1,0
881,4999230,0,1,1,0
882,4999848,1,1,0,0
883,4999872,0,0,1,0


In [27]:
features_v2 = features.merge(df_comorb, how='left', on='a_patientid')

for col in df_comorb.columns[1:]:
    features_v2[col] = features_v2[col].fillna(0)
    
features_v2

,a_patientid,min_SpO2,max_SpO2,min_Albumina,min_Hematòcrit,min_Potassi,min_Urea,min_pH,max_Albumina,max_Hematòcrit,max_Potassi,max_Urea,max_pH,diabetis_flag,ckd_flag,hyperTension_flag,heartFailure_flag
0,4000318,91.3,100.0,NaN,39.5,NaN,NaN,7.35,NaN,39.5,NaN,NaN,7.35,0.0,0.0,0.0,0.0
1,4000422,91.0,99.0,3.9,34.3,4.09,83.0,7.16,3.9,34.3,4.09,83.0,7.17,0.0,0.0,0.0,0.0
2,4000555,93.3,100.0,NaN,49.6,NaN,NaN,7.26,NaN,49.6,NaN,NaN,7.44,1.0,0.0,1.0,0.0
3,4000561,92.9,100.0,4.5,51.3,3.97,27.0,7.38,4.5,51.3,3.97,27.0,7.40,0.0,0.0,1.0,0.0
4,4000592,73.9,100.0,2.9,36.7,4.00,67.0,7.27,2.9,41.9,4.57,68.0,7.43,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3804,4999651,97.1,100.0,NaN,30.5,NaN,NaN,7.42,NaN,30.5,NaN,NaN,7.44,0.0,0.0,0.0,0.0
3805,4999848,89.1,100.0,NaN,28.5,4.63,97.0,7.32,NaN,32.6,4.92,100.0,7.34,1.0,1.0,0.0,0.0
3806,4999872,96.0,100.0,NaN,32.0,3.25,25.0,7.44,NaN,32.0,3.25,25.0,7.44,0.0,0.0,1.0,0.0
3807,4999880,94.0,98.0,2.9,24.1,5.61,122.0,7.34,3.1,26.6,5.80,127.0,7.34,0.0,0.0,0.0,0.0


In [28]:
target = pd.pivot_table(
    outcomes,
    index=['a_patientid','time'],
    columns=['a_variableid'],
    values='value'
).reset_index()

target = target.rename(columns={4000001380:'flux_sang', 4000005666:'flux_efluent'})
#tmp_path = './tmp/target.csv'
#target.to_csv(tmp_path)
#target = pd.read_csv(tmp_path, index_col=0)
target

a_variableid,a_patientid,time,flux_sang,flux_efluent
0,4002523,489,200.0,3002.0
1,4002523,534,200.0,3052.0
2,4002523,774,200.0,3052.0
3,4002523,834,200.0,3052.0
4,4002523,954,200.0,3052.0
...,...,...,...,...
135891,4996001,90079,100.0,202.0
135892,4996001,90094,100.0,202.0
135893,4996001,90109,100.0,202.0
135894,4996001,90124,100.0,202.0


In [33]:
target['Y'] = list(map(lambda x,y: 1 if x>=0 and y>=0 else 0, target.flux_sang.fillna(-1), target.flux_efluent.fillna(-1)))  
first_crrt = target.query('Y==1').groupby('a_patientid').time.min().reset_index()
u_target = target.merge(first_crrt, how='inner', on=['a_patientid','time'])

dataset = features_v2.merge(u_target.loc[:, ['a_patientid','time','Y']], how='left', on='a_patientid')

dataset.time = dataset.time.fillna(-1)
dataset.Y = dataset.Y.fillna(0)

#dataset_v2 = dataset.query("time<0 or time>=820")#.drop(columns=['time'])
dataset

,a_patientid,min_SpO2,max_SpO2,min_Albumina,min_Hematòcrit,min_Potassi,min_Urea,min_pH,max_Albumina,max_Hematòcrit,max_Potassi,max_Urea,max_pH,diabetis_flag,ckd_flag,hyperTension_flag,heartFailure_flag,time,Y
0,4000318,91.3,100.0,NaN,39.5,NaN,NaN,7.35,NaN,39.5,NaN,NaN,7.35,0.0,0.0,0.0,0.0,-1.0,0.0
1,4000422,91.0,99.0,3.9,34.3,4.09,83.0,7.16,3.9,34.3,4.09,83.0,7.17,0.0,0.0,0.0,0.0,-1.0,0.0
2,4000555,93.3,100.0,NaN,49.6,NaN,NaN,7.26,NaN,49.6,NaN,NaN,7.44,1.0,0.0,1.0,0.0,-1.0,0.0
3,4000561,92.9,100.0,4.5,51.3,3.97,27.0,7.38,4.5,51.3,3.97,27.0,7.40,0.0,0.0,1.0,0.0,-1.0,0.0
4,4000592,73.9,100.0,2.9,36.7,4.00,67.0,7.27,2.9,41.9,4.57,68.0,7.43,0.0,0.0,0.0,0.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3804,4999651,97.1,100.0,NaN,30.5,NaN,NaN,7.42,NaN,30.5,NaN,NaN,7.44,0.0,0.0,0.0,0.0,-1.0,0.0
3805,4999848,89.1,100.0,NaN,28.5,4.63,97.0,7.32,NaN,32.6,4.92,100.0,7.34,1.0,1.0,0.0,0.0,-1.0,0.0
3806,4999872,96.0,100.0,NaN,32.0,3.25,25.0,7.44,NaN,32.0,3.25,25.0,7.44,0.0,0.0,1.0,0.0,-1.0,0.0
3807,4999880,94.0,98.0,2.9,24.1,5.61,122.0,7.34,3.1,26.6,5.80,127.0,7.34,0.0,0.0,0.0,0.0,-1.0,0.0


In [34]:
dataset.isna().sum() / dataset.shape[0]

a_patientid          0.000000
min_SpO2             0.000000
max_SpO2             0.000000
min_Albumina         0.733001
min_Hematòcrit       0.095038
min_Potassi          0.417957
min_Urea             0.396692
min_pH               0.040431
max_Albumina         0.733001
max_Hematòcrit       0.095038
max_Potassi          0.417957
max_Urea             0.396692
max_pH               0.040431
diabetis_flag        0.000000
ckd_flag             0.000000
hyperTension_flag    0.000000
heartFailure_flag    0.000000
time                 0.000000
Y                    0.000000
dtype: float64